In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [35]:
## Load the dataset
data = pd.read_csv("Churn_Modelling.csv")

In [36]:
## preprocesss the data
### Drop irrevelant columns
data=data.drop(columns=["RowNumber","CustomerId","Surname"],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [37]:
## Encode Categorical Variables
le =LabelEncoder()
data["Gender"]=le.fit_transform(data["Gender"])

In [38]:
## Onehot encoding "Geography"
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo = ohe.fit_transform(data[["Geography"]])
geo

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [39]:
ohe.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [40]:
df = pd.DataFrame(geo.toarray(),columns=ohe.get_feature_names_out(["Geography"]))
df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [41]:
## Combine one hot encoded columns with the original data
data = pd.concat([data.drop("Geography",axis=1),df],axis=1)

In [42]:
## Save the encoders and scaler
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(le,file)
with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(ohe,file)

In [43]:
## Divide the dataset into dependent and dependent features
x = data.drop("Exited",axis=1)
y = data["Exited"]



In [44]:

## Split the data in training and testing data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)


## Scale these features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [45]:
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)
    

# ANN IMPLEMENTATION

In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [47]:
## Build Our ANN Model
model = Sequential([
    Dense(64,activation="relu",input_shape=(x_train.shape[1],)), ##HL 1 Connected with input layer
    Dense(32,activation="relu"), #HL2
    Dense(1,activation="sigmoid") ## Output layer

])

c:\Users\vigna\Desktop\DLPROJECT1\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [48]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x000002A4787EB560>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [50]:
## Compile the model
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [51]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = "logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [52]:
## Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor = "val_loss",patience=5,restore_best_weights=True)



In [53]:
### Train the model
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,
                    callbacks=[tensorflow_callback,early_stopping_callback])


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8114 - loss: 0.4436 - val_accuracy: 0.8345 - val_loss: 0.3899
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8457 - loss: 0.3789 - val_accuracy: 0.8505 - val_loss: 0.3618
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8574 - loss: 0.3530 - val_accuracy: 0.8565 - val_loss: 0.3511
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8591 - loss: 0.3449 - val_accuracy: 0.8585 - val_loss: 0.3459
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8579 - loss: 0.3407 - val_accuracy: 0.8595 - val_loss: 0.3445
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8611 - loss: 0.3384 - val_accuracy: 0.8585 - val_loss: 0.3481
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8612 - loss: 0.3351 - val_accuracy: 0.8580 - val_loss: 0.3451
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8629 - loss: 0.3307 - val_accu

In [56]:
model.save("model.keras")


In [ ]:
## Load the Tensorboard Extension
%load_ext tensorboard


In [ ]:
%tensorboard --logdir logs/fit20251210-103043

Reusing TensorBoard on port 6007 (pid 30392), started 0:02:33 ago. (Use '!kill 30392' to kill it.)

In [ ]:
import os
os.listdir("logs/fit20251210-103043")


['train', 'validation']